In [ ]:
import requests
import pandas as pd
import hvplot.pandas
import json
import holoviews as hv

hv.extension('bokeh')

In [ ]:
# Define the base URL for the API
base_url = "https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/api/Group/"

# Fetch the list of providers
providers_response = requests.get(base_url)
providers_data = json.loads(providers_response.text)  # Parse the JSON response
providers_data

In [ ]:

# Initialize a list to hold all station data
all_stations_data = []

# Loop through the list of providers and fetch their stations
for provider in providers_data:
    provider_name = provider['Name']
    stations_url = base_url + provider_name + "/Devices"
    stations_response = requests.get(stations_url)
    stations_data = json.loads(stations_response.text)  # Parse the JSON response
    
    # Loop through each station and extract data
    for device in stations_data:
        station_data = {
            'Provider': provider_name,
            'Id': device['Id'],
            'Name': device['Name'],
            'lat': device['Lat'],
            'lon': device['Lon'],
            'LastAccessStatus': device['CurrentStatus']['LastAccessStatus'],
            'LastDate': device['CurrentStatus']['LastDate'],
            'State': device['CurrentStatus']['State'],
            'SyncStatus': device['CurrentStatus']['SyncStatus'],
            'FileType': device['FileType'],
            'GroupId': device['GroupId'],
            'MovAvgNp': device['MovAvgNp'],
            'Notes': device.get('Notes'),  # Use .get() to handle missing keys
            'Source': device['Source']
            # Add other fields as necessary
        }
        all_stations_data.append(station_data)

df = pd.DataFrame(all_stations_data)
df

In [ ]:
df.groupby(['Provider','State']).count()[['Id']].hvplot.bar(
    rot = 90,
    logy = True,
    ylim = [1, 2000], 
    grid = True
).opts(
    width = 1200,
    height = 800,
    title = "data availability"
)

In [ ]:
df.hvplot.points(
    x = 'lon',
    y='lat',
    c = 'Provider',
    line_color = 'k',
    cmap = 'glasbey',
    geo = True,
    tiles="EsriImagery",
).opts(
    width = 1200,
    height = 800
)